In [1]:
import urllib.request
import urllib.error
import time
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
from datetime import datetime
import time
import tqdm

headers = {'User-Agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

In [45]:
r = requests.get('https://sugang.korea.ac.kr',headers= headers)
c = r.cookies
ms = c.get_dict()['COOK_TS']
cookies = c
d = c.get_dict()
Year = '2022'
Semester = '1R'

# 대학 코드 
payload ={
"obj": "pCol",
"args": ["KOR",Year,Semester,"1","00"]
}

header = {'Referer':'https://sugang.korea.ac.kr/core?attribute=coreMain&pWname=null&flagx=X&fake='+str(ms)}
res = requests.post('https://sugang.korea.ac.kr/view?attribute=combo&fake='+str(ms),headers= header,
                     cookies=d,data = payload)
college_code = [i['code'] for i in res.json()['data']]


# 교양 코드
payload ={
'obj':"pGroupCd",
"args": ["KOR",Year,Semester]
}

header = {'Referer':'https://sugang.korea.ac.kr/core?attribute=coreMain&pWname=null&flagx=X&fake='+str(ms)}
res = requests.post('https://sugang.korea.ac.kr/view?attribute=combo&fake='+str(ms),headers= header,
                     cookies=d,params = payload)

general_code =  res.json()['data']




all_dep_code = {}


# 전공 코드(대학 코드로부터 )
for college in college_code:
    
    payload ={"obj": "pDept",
               "args": ["KOR",college,Year,Semester,"00"]}
    
    header = {'Referer':
              'https://sugang.korea.ac.kr/core?attribute=coreMain&pWname=null&flagx=X&fake='+
              str(ms)}
    
    res = requests.post('https://sugang.korea.ac.kr/view?attribute=combo&fake='+str(ms),headers= header,
                         cookies=d, data = payload)

    all_dep_code[college] = [i['code'] for i in res.json()['data']]
    

final_lst = []

parts = ['00','24','01','30']
for part in tqdm.tqdm(parts): # 00 : 전공 , 24 : 학문의기초, 01 : 교양, 30 : 교직
    
    if (part == '00') or (part == '24'):
        
        
        
        for col in all_dep_code:
            for dep in all_dep_code[col]:
                        
                payload_1 = {
                'pYear': Year,
                'pTerm': Semester,
                'pCampus': '1', # 안암 : 1 , 세종 : 2
                'pGradCd': '0136', # 대학 : 0136, 대학원 : 0309
                'pCourDiv': part, # 전공 : 00 , 학문의 기초 : 24, 교양 : 01, 교직 ..... # 학과별로 있는 것은 모두 00으로 표현.
                'pCol': col,  # College
                'pDept': dep, # Department
                'pCredit': "", 
                'pDay': "" ,
                'pStartTime': "", 
                'pEndTime': "", 
                'pProf': "",
                'pCourCd': "",
                'pCourNm': "", 
                'strYear': Year,
                'strTerm': Semester,
                'strUserType': "......",
                'strChasu': ""
                }
                

                header = {'Referer':'https://sugang.korea.ac.kr/core?attribute=coreMain&pWname=null&flagx=X&fake='+str(ms)}
                res_0 = requests.post('https://sugang.korea.ac.kr/view?attribute=lectHakbuData&fake='+str(ms),headers= header,
                                     cookies=d,data =payload_1)
                final_lst.append(res_0.json())
                
    elif part == '01':
        
        for code in general_code:

            
            payload_2 = {
            "pYear": Year,
            "pTerm": Semester,
            "pCampus": "1",
            "pGradCd": "0136", # 0136 : 대학 , 다른건 대학원
            "pCourDiv": part, # 교양, 전공, 등등 구분 번호
            "pGroupCd": code['code'], # 교양만 구분 번호
            "pCredit": "",
            "pDay": "",
            "pStartTime": "",
            "pEndTime": "",
            "pProf": "",
            "pCourCd": "",
            "pCourNm": "",
            "strYear": Year,
            "strTerm": Semester,
            "strUserType": "......",
            "strChasu": ""
                }

            header = {'Referer':'https://sugang.korea.ac.kr/core?attribute=coreMain&pWname=null&flagx=X&fake='+str(ms)}
            res_0 = requests.post('https://sugang.korea.ac.kr/view?attribute=lectHakbuData&fake='+str(ms),headers= header,
                                 cookies=d,data =payload_2)
            final_lst.append(res_0.json())
         
        
            
    elif part == '30':
        payload_3 ={
        "pYear": Year,
        "pTerm": Semester,
        "pCampus": "1",
        "pGradCd": "0136", # 0136 : 대학 , 다른건 대학원
        "pCourDiv": part, # 교양, 전공, 등등 구분 번호
        "pCredit": "",
        "pDay": "",
        "pStartTime": "",
        "pEndTime": "",
        "pProf": "",
        "pCourCd": "",
        "pCourNm": "",
        "strYear": Year,
        "strTerm": Semester,
        "strUserType": "......",
        "strChasu": ""
            }
        
        header = {'Referer':'https://sugang.korea.ac.kr/core?attribute=coreMain&pWname=null&flagx=X&fake='+str(ms)}
        res_0 = requests.post('https://sugang.korea.ac.kr/view?attribute=lectHakbuData&fake='+str(ms),headers= header,
                             cookies=d,data =payload_3)
        
        final_lst.append(res_0.json())
        
for i in range(len(final_lst)):
    if i == 0:
        df2 = pd.DataFrame(final_lst[i]['data'])
    
    df1 = pd.DataFrame(final_lst[i]['data'])
    
    df2 = pd.concat([df2,df1],axis=0)

df3 = df2[['year','isu_nm',
         'cour_div', 'term'
       , 'department' ,'time_room',
       'campus', 'prof_nm', 'waiting_yn',
     'cour_nm', 'dept_cd', 'cour_cls', 'time', 'apply_dept',
       ]]

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.87s/it]


In [47]:
df3

,year,isu_nm,cour_div,term,department,time_room,campus,prof_nm,waiting_yn,cour_nm,dept_cd,cour_cls,time,apply_dept
0,2022,전공필수,11,1R,간호학과,화(3) 우정간호학관 307호,서울,송준아,N,간호학개론,0233,00,1(1),
1,2022,전공선택,10,1R,간호학과,수(1-2) 우정간호학관 101호,서울,김경해,N,건강과식이(영강),0233,00,2(2),
2,2022,전공선택,10,1R,간호학과,월(3-4) 우정간호학관 101호,서울,김이랑,N,간호정보학및실습,0233,00,2(2),
3,2022,전공선택,10,1R,간호학과,화(1-2) 우정간호학관 101호,서울,설근희,N,기초간호과학II,0233,00,2(2),
4,2022,전공선택,10,1R,간호학과,화(5-6) 우정간호학관 101호,서울,신유경,N,기초간호과학III,0233,00,3(3),
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2022,교직,33,1R,교직팀,목(3-4) 사대신관 124호,서울,양준영,N,학교폭력예방및학생의이해,5960,06,2(2),*Course not open for foreign exchange students
58,2022,교직,33,1R,교직팀,토(2),서울,박인우,N,학교현장실습,5960,01,2(2),*Course not open for foreign exchange students
59,2022,교직,33,1R,교직팀,토(2),서울,신창호,N,학교현장실습,5960,02,2(2),*Course not open for foreign exchange students
60,2022,교직,33,1R,교직팀,토(2),서울,손승현,N,학교현장실습,5960,03,2(2),*Course not open for foreign exchange students
